<a href="https://colab.research.google.com/github/Tung0000/MobileNet_ssd/blob/main/mobile_ssd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!pip list

In [ ]:
!pip install numpy
!pip install opencv-python

In [ ]:
import numpy as np
import cv2
from google.colab.patches import cv2_imshow   # สำหรับแสดงภาพใน Colab

# รายชื่อหมวดหมู่ทั้งหมด
CLASSES = ["BACKGROUND", "AEROPLANE", "BICYCLE", "BIRD", "BOAT",
           "BOTTLE", "BUS", "CAR", "CAT", "CHAIR", "COW", "DININGTABLE",
           "DOG", "HORSE", "MOTORBIKE", "PERSON", "POTTEDPLANT", "SHEEP",
           "SOFA", "TRAIN", "TVMONITOR"]

# สีสุ่มสำหรับวาดกรอบ
COLORS = np.random.uniform(0, 100, size=(len(CLASSES), 3))


def load_model(proto_path, model_path):
    """โหลดโมเดลจากไฟล์ prototxt และ caffemodel"""
    net = cv2.dnn.readNetFromCaffe(proto_path, model_path)
    return net


def process_frame(frame, net, conf_threshold=0.5, draw=True):
    """ประมวลผลทีละเฟรมและคืนค่าเฟรม + results"""
    (h, w) = frame.shape[:2]
    results = []

    blob = cv2.dnn.blobFromImage(frame, 0.007843, (300, 300), 127.5)
    net.setInput(blob)
    detections = net.forward()

    for i in np.arange(0, detections.shape[2]):
        percent = detections[0, 0, i, 2]
        if percent > conf_threshold:
            class_index = int(detections[0, 0, i, 1])
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")

            label = "{} [{:.2f}%]".format(CLASSES[class_index], percent * 100)
            results.append({
                "class": CLASSES[class_index],
                "confidence": float(percent),
                "box": (int(startX), int(startY), int(endX), int(endY))
            })

            if draw:
                cv2.rectangle(frame, (startX, startY), (endX, endY), COLORS[class_index], 2)
                cv2.rectangle(frame, (startX - 1, startY - 30), (endX + 1, startY),
                              COLORS[class_index], cv2.FILLED)
                y = startY - 15 if startY - 15 > 15 else startY + 15
                cv2.putText(frame, label, (startX + 20, y + 5),
                            cv2.FONT_HERSHEY_DUPLEX, 0.6, (255, 255, 255), 1)

    return frame, results


def run_image(net, image_path):
    """ตรวจจับวัตถุจากรูปภาพ"""
    frame = cv2.imread(image_path)
    if frame is None:
        raise FileNotFoundError(f"ไม่พบไฟล์รูปภาพ: {image_path}")

    frame, results = process_frame(frame, net)
    cv2_imshow(frame)  # ใช้แทน cv2.imshow
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    return results


def run_video(net, video_path):
    """ตรวจจับวัตถุจากวิดีโอ (แสดงทีละเฟรมใน Colab)"""
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        raise FileNotFoundError(f"ไม่พบไฟล์วิดีโอ: {video_path}")

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame, results = process_frame(frame, net)
        cv2_imshow(frame)  # ใช้แทน cv2.imshow

        if results:
            print(results)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()


# ------------------ Main ------------------
if __name__ == "__main__":
    net = load_model("MobileNetSSD_deploy.prototxt",
                     "MobileNetSSD_deploy.caffemodel")

    results = run_image(net, "Coco.png")     # ใช้รูปภาพ
    run_video(net, "test.mp4")               # ใช้วิดีโอ

    print("Final Results:", results)